# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  May 31 2022  1:23PM  238450        10        6528454   
1  May 31 2022  1:17PM  238449        10        SO 2783   
2  May 31 2022  1:13PM  238448       102  PBR-052022-01   
3  May 31 2022 12:54PM  238446        20    ACI-8037719   
4  May 31 2022 12:54PM  238446        20        8072432   
5  May 31 2022 12:54PM  238446        20        8081984   
6  May 31 2022 12:52PM  238445        15        8071834   
7  May 31 2022 12:52PM  238445        15        8071835   
8  May 31 2022 12:48PM  238442        20        8053707   
9  May 31 2022 12:48PM  238442        20        8070513   

                   Customer ShipmentStatus  
0  Senseonics, Incorporated       Released  
1         Yusen – 3D Matrix       Released  
2  Senseonics, Incorporated       Released  
3  ACI Healthcare USA, Inc.       Released  
4  ACI Healthcare USA, Inc.       Released  
5  ACI Healthcare USA, Inc.       Released  
6    Mizner Bioscience, LLC       Released  
7    Mizner Bioscience, LLC       Released  
8  ACI Healthcare USA, Inc.       Released  
9  ACI Healthcare USA, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
48  238445       Released          2
49  238446       Released          3
50  238448       Released          1
51  238449       Released          1
52  238450       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238445                NaN        NaN       2.0
238446                NaN        NaN       3.0
238448                NaN        NaN       1.0
238449                NaN        NaN       1.0
238450                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238317                0.0        9.0       1.0
238334                1.0        9.0       4.0
238341                0.0        0.0      35.0
238347                0.0        0.0      36.0
238349               11.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238317                  0          9         1
238334                  1          9         4
238341                  0          0        35
238347                  0          0        36
238349                 11          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238317          0          9         1
1               238334          1          9         4
2               238341          0          0        35
3               238347          0          0        36
4               238349         11          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               238317                   9        1
1               238334         1         9        4
2               238341                           35
3               238347                           36
4               238349        11                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   May 31 2022  1:23PM  238450        10   
1   May 31 2022  1:17PM  238449        10   
2   May 31 2022  1:13PM  238448       102   
3   May 31 2022 12:54PM  238446        20   
6   May 31 2022 12:52PM  238445        15   
8   May 31 2022 12:48PM  238442        20   
25  May 31 2022 12:43PM  238440        10   
38  May 31 2022 12:42PM  238439        15   
39  May 31 2022 12:35PM  238435        15   
40  May 31 2022 12:26PM  238430        16   

                                 Customer  
0                Senseonics, Incorporated  
1                       Yusen – 3D Matrix  
2                Senseonics, Incorporated  
3                ACI Healthcare USA, Inc.  
6                  Mizner Bioscience, LLC  
8                ACI Healthcare USA, Inc.  
25                       Methapharm, Inc.  
38             Laser Pharmaceuticals, LLC  
39  Carwin Pharmaceutical Associates, LLC  
40         Sartorius Bioprocess Solutions

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  May 31 2022  1:23PM  238450        10   
1  May 31 2022  1:17PM  238449        10   
2  May 31 2022  1:13PM  238448       102   
3  May 31 2022 12:54PM  238446        20   
4  May 31 2022 12:52PM  238445        15   
5  May 31 2022 12:48PM  238442        20   
6  May 31 2022 12:43PM  238440        10   
7  May 31 2022 12:42PM  238439        15   
8  May 31 2022 12:35PM  238435        15   
9  May 31 2022 12:26PM  238430        16   

                                Customer Completed Executing Released  
0               Senseonics, Incorporated                            1  
1                      Yusen – 3D Matrix                            1  
2               Senseonics, Incorporated                            1  
3               ACI Healthcare USA, Inc.                            3  
4                 Mizner Bioscience, LLC                            2  
5               ACI Healthcare USA, Inc.                           17  
6                       Methapharm, Inc.                   8        5  
7             Laser Pharmaceuticals, LLC                            1  
8  Carwin Pharmaceutical Associates, LLC                            1  
9         Sartorius Bioprocess Solutions                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  May 31 2022  1:23PM  238450        10   
1  May 31 2022  1:17PM  238449        10   
2  May 31 2022  1:13PM  238448       102   
3  May 31 2022 12:54PM  238446        20   
4  May 31 2022 12:52PM  238445        15   
5  May 31 2022 12:48PM  238442        20   
6  May 31 2022 12:43PM  238440        10   
7  May 31 2022 12:42PM  238439        15   
8  May 31 2022 12:35PM  238435        15   
9  May 31 2022 12:26PM  238430        16   

                                Customer Released Executing Completed  
0               Senseonics, Incorporated        1                      
1                      Yusen – 3D Matrix        1                      
2               Senseonics, Incorporated        1                      
3               ACI Healthcare USA, Inc.        3                      
4                 Mizner Bioscience, LLC        2                      
5               ACI Healthcare USA, Inc.       17                      
6                       Methapharm, Inc.        5         8            
7             Laser Pharmaceuticals, LLC        1                      
8  Carwin Pharmaceutical Associates, LLC        1                      
9         Sartorius Bioprocess Solutions        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  May 31 2022  1:23PM  238450        10  Senseonics, Incorporated        1   
1  May 31 2022  1:17PM  238449        10         Yusen – 3D Matrix        1   
2  May 31 2022  1:13PM  238448       102  Senseonics, Incorporated        1   
3  May 31 2022 12:54PM  238446        20  ACI Healthcare USA, Inc.        3   
4  May 31 2022 12:52PM  238445        15    Mizner Bioscience, LLC        2   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  May 31 2022  1:23PM  238450        10  Senseonics, Incorporated       1.0   
1  May 31 2022  1:17PM  238449        10         Yusen – 3D Matrix       1.0   
2  May 31 2022  1:13PM  238448       102  Senseonics, Incorporated       1.0   
3  May 31 2022 12:54PM  238446        20  ACI Healthcare USA, Inc.       3.0   
4  May 31 2022 12:52PM  238445        15    Mizner Bioscience, LLC       2.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  May 31 2022  1:23PM  238450        10  Senseonics, Incorporated       1.0   
1  May 31 2022  1:17PM  238449        10         Yusen – 3D Matrix       1.0   
2  May 31 2022  1:13PM  238448       102  Senseonics, Incorporated       1.0   
3  May 31 2022 12:54PM  238446        20  ACI Healthcare USA, Inc.       3.0   
4  May 31 2022 12:52PM  238445        15    Mizner Bioscience, LLC       2.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         5006428     454.0       14.0       11.0
102         715297       2.0        1.0        0.0
12          715051      76.0        0.0        0.0
15          953636       5.0        9.0        0.0
16          238430       2.0        0.0        0.0
18          953578       4.0        0.0        0.0
19          476781       3.0       12.0        0.0
20         1430473      69.0        9.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  5006428     454.0       14.0       11.0
1       102   715297       2.0        1.0        0.0
2        12   715051      76.0        0.0        0.0
3        15   953636       5.0        9.0        0.0
4        16   238430       2.0        0.0        0.0
5        18   953578       4.0        0.0        0.0
6        19   476781       3.0       12.0        0.0
7        20  1430473      69.0        9.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     454.0       14.0       11.0
1       102       2.0        1.0        0.0
2        12      76.0        0.0        0.0
3        15       5.0        9.0        0.0
4        16       2.0        0.0        0.0
5        18       4.0        0.0        0.0
6        19       3.0       12.0        0.0
7        20      69.0        9.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  454.0
1       102  Released    2.0
2        12  Released   76.0
3        15  Released    5.0
4        16  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102    12   15   16   18    19    20
Status                                                
Completed   11.0  0.0   0.0  0.0  0.0  0.0   0.0   1.0
Executing   14.0  1.0   0.0  9.0  0.0  0.0  12.0   9.0
Released   454.0  2.0  76.0  5.0  2.0  4.0   3.0  69.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102    12   15   16   18    19    20
0          Completed   11.0  0.0   0.0  0.0  0.0  0.0   0.0   1.0
1          Executing   14.0  1.0   0.0  9.0  0.0  0.0  12.0   9.0
2           Released  454.0  2.0  76.0  5.0  2.0  4.0   3.0  69.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102    12   15   16   18    19    20
0  Completed   11.0  0.0   0.0  0.0  0.0  0.0   0.0   1.0
1  Executing   14.0  1.0   0.0  9.0  0.0  0.0  12.0   9.0
2   Released  454.0  2.0  76.0  5.0  2.0  4.0   3.0  69.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()